In [1]:
import pandas as pd

# load data
df = pd.read_csv("CLEAN_DATA/Data_europe_2005_2020_ps.csv")
df


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,Unnamed: 0,country,countryname,oecdmember,eumember,edate,date,party,partyname,partyabbrev,...,welfare,intpeace,datasetversion,id_perm,year,Gini coefficient,welfare spending,electoral_dem_ind,GNI,stance_positivity_score
0,0,11,Sweden,10,10.0,2006-09-17,200609,11110,Green Ecology Party,MP,...,29.798,1.515,2023a,4REHJD,2006,0.263525,26.408,0.915,4.340743e+11,35.352
1,1,11,Sweden,10,10.0,2006-09-17,200609,11220,Left Party,V,...,24.679,1.028,2023a,PNMPGP,2006,0.263525,26.408,0.915,4.340743e+11,33.674
2,2,11,Sweden,10,10.0,2006-09-17,200609,11320,Social Democratic Labour Party,SAP,...,29.517,1.781,2023a,K3SXQ7,2006,0.263525,26.408,0.915,4.340743e+11,34.606
3,3,11,Sweden,10,10.0,2006-09-17,200609,11420,Liberal People’s Party,FP,...,16.694,0.000,2023a,VSD37H,2006,0.263525,26.408,0.915,4.340743e+11,34.381
4,4,11,Sweden,10,10.0,2006-09-17,200609,11520,Christian Democrats,Kd,...,20.968,0.000,2023a,H41DX2,2006,0.263525,26.408,0.915,4.340743e+11,34.676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,1124,98,Ukraine,0,0.0,2019-07-21,201907,98350,Opposition Bloc,NaN,...,13.208,12.264,2023a,ZHUFI7,2019,0.266195,NaN,0.524,1.557951e+11,62.262
1125,1125,98,Ukraine,0,0.0,2019-07-21,201907,98440,Servant of the People,NaN,...,4.255,0.000,2023a,RB6257,2019,0.266195,NaN,0.524,1.557951e+11,62.767
1126,1126,98,Ukraine,0,0.0,2019-07-21,201907,98450,Voice,NaN,...,3.614,9.639,2023a,JT5WL6,2019,0.266195,NaN,0.524,1.557951e+11,48.192
1127,1127,98,Ukraine,0,0.0,2019-07-21,201907,98617,All-Ukrainian Union ‘Fatherland',NaN,...,15.584,5.195,2023a,W96R9B,2019,0.266195,NaN,0.524,1.557951e+11,70.129


In [2]:
# Filter the DataFrame
df = df[['edate', 'countryname', 'rile', 'stance_positivity_score', 'Gini coefficient', 'welfare spending', 'electoral_dem_ind', 'GNI', 'absseat']]

# Convert 'date' to datetime and extract the year for grouping
df['edate'] = pd.to_datetime(df['edate'])
df['year'] = df['edate'].dt.year

# Calculate mean values for each group
mean_values = df.groupby(['countryname', 'year']).transform('mean')

# Since transform keeps the original index, copy the mean values directly into the original df
df['mean_rile'] = mean_values['rile']
df['mean_stance_positivity_score'] = mean_values['stance_positivity_score']
df['mean_gini_coefficient'] = mean_values['Gini coefficient']

# Now, calculate 'leading' for each group
# First, mark each row as 'left' or 'right' based on 'rile'
df['side'] = df['rile'].apply(lambda x: 'left' if x < 0 else 'right')

# Initialize an empty column for 'leading'
df['leading'] = None

# Iterate through each group to determine the leading side
for (countryname, year), group in df.groupby(['countryname', 'year']):
    left_sum = group[group['side'] == 'left']['absseat'].sum()
    right_sum = group[group['side'] == 'right']['absseat'].sum()
    leading_side = 'left' if left_sum > right_sum else 'right'
    
    # Assign the 'leading' side to the original df
    df.loc[(df['countryname'] == countryname) & (df['year'] == year), 'leading'] = leading_side

# Drop duplicates since each country-year group now has the same 'leading' value and mean calculations
final_df = df[['countryname', 'year', 'mean_rile', 'mean_stance_positivity_score', 'mean_gini_coefficient', 'leading', 'welfare spending', 'electoral_dem_ind', 'GNI']].drop_duplicates()

# Optionally, rename the columns back
final_df.columns = ['countryname', 'year', 'rile', 'stance_positivity_score', 'gini Coefficient', 'leading', 'welfare spending', 'electoral_dem_ind', 'GNI']

# Display the final DataFrame
display(final_df.head())

/var/folders/bb/hkfpdvbn46vcqvd2x75jvk0h0000gn/T/ipykernel_12921/547183562.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['edate'] = pd.to_datetime(df['edate'])
/var/folders/bb/hkfpdvbn46vcqvd2x75jvk0h0000gn/T/ipykernel_12921/547183562.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['edate'].dt.year
/var/folders/bb/hkfpdvbn46vcqvd2x75jvk0h0000gn/T/ipykernel_12921/547183562.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,countryname,year,rile,stance_positivity_score,gini Coefficient,leading,welfare spending,electoral_dem_ind,GNI
0,Sweden,2006,-8.132000,34.300000,0.263525,left,26.408,0.915,4.340743e+11
7,Sweden,2010,-4.296375,34.593625,0.277382,left,25.849,0.915,5.092813e+11
15,Sweden,2014,-15.754750,54.500250,0.283897,left,26.587,0.916,5.942473e+11
23,Sweden,2018,-10.283625,58.764000,0.299742,left,25.633,0.909,5.665538e+11
31,Norway,2005,-15.257286,37.567143,0.305976,left,20.737,0.892,3.133711e+11


In [3]:
#save as csv 
final_df.to_csv("CLEAN_DATA/2005-2020_percountry_peryear.csv")